In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
import imutils
#from FaceDetector import *
from keras.models import model_from_json
import keras
from data_generator import *
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import time
from parameters import *

Using TensorFlow backend.


In [23]:
def triplet_loss(y_true, y_pred, alpha = ALPHA):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    return loss

In [24]:
best_model_path=None
if os.path.exists("./bestmodel.txt"):
    with open('bestmodel.txt', 'r') as file:
        best_model_path = file.read()
    
if best_model_path != None and os.path.exists(best_model_path):
    print("Pre trained model found")
    FRmodel = keras.models.load_model(best_model_path, custom_objects={'triplet_loss': triplet_loss})
    
else:
    print('Saved model not found, loading untrained FaceNet')
    FRmodel = faceRecoModel(input_shape=(3, IMAGE_SIZE, IMAGE_SIZE))
    load_weights_from_FaceNet(FRmodel)

for layer in FRmodel.layers[0: LAYERS_TO_FREEZE]:
    layer.trainable  =  False

Pre trained model found


In [25]:
input_shape=(3, IMAGE_SIZE, IMAGE_SIZE)
A = Input(shape=input_shape, name = 'anchor')
P = Input(shape=input_shape, name = 'anchorPositive')
N = Input(shape=input_shape, name = 'anchorNegative')

In [26]:
enc_A = FRmodel(A)
enc_P = FRmodel(P)
enc_N = FRmodel(N)

In [27]:
# Callbacks
early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=0.00005)
STAMP = 'facenet_%d'%(1) 
checkpoint_dir = './' + 'checkpoints/' + str(int(time.time())) + '/'

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [28]:
bst_model_path = checkpoint_dir + STAMP + '.h5'
tensorboard = TensorBoard(log_dir=checkpoint_dir + "logs/{}".format(time.time()))

In [30]:
# Model
tripletModel = Model(inputs=[A, P, N], outputs=[enc_A, enc_P, enc_N])
tripletModel.compile(optimizer = 'adam', loss = triplet_loss)

In [31]:
gen = batch_generator(BATCH_SIZE)
tripletModel.fit_generator(gen, epochs=NUM_EPOCHS,steps_per_epoch=STEPS_PER_EPOCH,callbacks=[early_stopping, tensorboard])

Epoch 1/10
5/5 [==============================] - 74s 15s/step - loss: 1.9652 - FaceRecoModel_loss: 0.6053
Epoch 2/10
5/5 [==============================] - 37s 7s/step - loss: 1.5239 - FaceRecoModel_loss: 0.5710
Epoch 3/10
5/5 [==============================] - 36s 7s/step - loss: 1.3469 - FaceRecoModel_loss: 0.4926
Epoch 4/10
5/5 [==============================] - 34s 7s/step - loss: 1.4286 - FaceRecoModel_loss: 0.5144
Epoch 5/10
5/5 [==============================] - 41s 8s/step - loss: 2.0217 - FaceRecoModel_loss: 0.5114
Epoch 6/10
5/5 [==============================] - 48s 10s/step - loss: 1.3373 - FaceRecoModel_loss: 0.5471
Epoch 7/10
5/5 [==============================] - 30s 6s/step - loss: 1.4901 - FaceRecoModel_loss: 0.6130
Epoch 8/10
5/5 [==============================] - 30s 6s/step - loss: 1.8674 - FaceRecoModel_loss: 0.6873
Epoch 9/10
5/5 [==============================] - 30s 6s/step - loss: 1.5756 - FaceRecoModel_loss: 0.5022
Epoch 10/10
5/5 [===========================